In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Objective

What happens to performance when we add the features from the original dataset?

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))

In [3]:
from transform_for_num_issues_pred import main as transform_dataset

In [4]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20, 5

from helper_functions import dummify_cols_and_baselines, make_alphas

In [5]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

In [6]:
df_transformed = transform_dataset(df_orig)
df_transformed.shape

../preprocessing/transform_for_num_issues_pred.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_subset['NUM_ISSUES'] = 1


(822, 27)

In [7]:
df_transformed.head(1).T

,0
tract_and_block_group,1004002
race_white,0.242399
race_black,0.514358
race_asian,0.035473
race_hispanic,0.0675676
race_other,0.140203
poverty_pop_below_poverty_level,0
earned_income_per_capita,34340
poverty_pop_w_public_assistance,0.0597484
poverty_pop_w_food_stamps,0.138365


## Choosing columns

In [8]:
cols_orig_dataset = ['NUM_ISSUES', 'tract_and_block_group']
cols_census = ['race_white',
     'race_black',
     'race_asian',
     'race_hispanic',
     'race_other',
     'poverty_pop_below_poverty_level',
     'earned_income_per_capita',
     'poverty_pop_w_public_assistance',
     'poverty_pop_w_food_stamps',
     'poverty_pop_w_ssi',
     'school',
     'school_std_dev',
     'housing',
     'housing_std_dev',
     'bedroom',
     'bedroom_std_dev',
     'value',
     'value_std_dev',
     'rent',
     'rent_std_dev',
     'income',
     'income_std_dev']

In [9]:
df_col_subset = df_transformed[cols_orig_dataset + cols_census]
df_col_subset.shape

(822, 24)

In [10]:
df = df_col_subset.dropna().drop_duplicates()
df.shape

(558, 24)

## Removing outliers

0303003 is City Hall, which is where issues are assigned to when they don't have a location. Ideally, I would find out which issues truly took place in that block group and filter accordingly.

The other Census block groups I chose from looking at a Tableau cloropleth map. They are places with few, if any people living there, like parks and in the financial distinct.

In [11]:
outliers = ('9818001', '9811003', '9807001', '0303003', '0701018')

In [12]:
df = df[~df.tract_and_block_group.isin(outliers)].drop('tract_and_block_group', axis=1)
df.shape

(553, 23)

In [15]:
df_transformed.NUM_ISSUES.describe()

count      822.000000
mean      1952.018248
std       5419.910281
min         50.000000
25%        582.250000
50%        833.000000
75%       1219.000000
max      29667.000000
Name: NUM_ISSUES, dtype: float64

In [16]:
df.NUM_ISSUES.describe()

count     553.000000
mean      874.059675
std       461.379271
min        50.000000
25%       580.000000
50%       807.000000
75%      1101.000000
max      3007.000000
Name: NUM_ISSUES, dtype: float64

## Dummify

In [17]:
cols_to_dummify = df.dtypes[df.dtypes == object].index
cols_to_dummify

Index([u'school', u'housing'], dtype='object')

In [18]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify)

8_6th_grade is baseline 0 2
rent is baseline 1 2


In [19]:
df_dummified.shape

(553, 32)

In [ ]:
# df_dummified = df_dummified.drop('race_other', axis=1)

## Running model

In [20]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import make_scorer


/Users/datascientist/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [40]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop('NUM_ISSUES', axis=1), 
    df_dummified.NUM_ISSUES, 
    test_size=0.2, 
    random_state=500
)

In [41]:
pipe = make_pipeline(StandardScaler(), LassoCV())

In [ ]:
mse = make_scorer(mean_squared_error, greater_is_better=False)

In [42]:
params = {'lassocv__alphas': make_alphas(-5, 5)}
params = {'lassocv__alphas': [[3], [10], [30]]}
model = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=10)
model.fit(X_train, y_train);

In [43]:
pd.DataFrame(model.cv_results_).T.iloc[:5]

,0,1,2
mean_fit_time,0.0416473,0.0323391,0.0239318
mean_score_time,0.00138454,0.000985503,0.00100396
mean_test_score,0.442863,0.451715,0.437078
mean_train_score,0.508133,0.494915,0.464107
param_lassocv__alphas,[3],[10],[30]


In [44]:
model.score(X_test, y_test)

0.30662264833319364

### Interesting coef values?

In [ ]:
coef_values = pd.DataFrame({
    'name': X_train.columns,
    'coef': model.best_estimator_.steps[-1][-1].coef_
})

coef_values['abs_coef'] = pd.np.abs(coef_values.coef)

In [ ]:
coef_values[coef_values.coef != 0].sort_values('abs_coef', ascending=False)

## Conclusion

Why does the model perform better when we _don't_ transform the # of 311 issues to per capita? 

One guess is that in areas where there are a high number of issues and low population, like the Financial District, transforming _distorts_ the # of 311 issues. The best denominator would be # of people in the area at 1pm, instead of # of people living there.

And perhaps the distribution of population amongst the Census block groups is fairly Uniform? I believe that's how they're decided.